## Deliverable 2. Create a Customer Travel Destinations Map.

In [4]:
# Dependencies and Setup
import pandas as pd
import gmaps
import requests

# Import API key
from config import g_key

# Configure gmaps API key
#gmaps.configure(api_key=g_key)

In [5]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Terrace,CA,54.5163,-128.6035,66.11,77,75,21.85,light rain
1,1,Zabid,YE,14.1951,43.3152,87.51,51,100,13.27,overcast clouds
2,2,Faanui,PF,-16.4833,-151.7500,79.03,70,24,10.29,few clouds
3,3,Kapaa,US,22.0752,-159.3190,82.72,75,90,5.01,light rain
4,4,Khatanga,RU,71.9667,102.5000,57.31,45,57,9.42,broken clouds


In [6]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 85


In [7]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Faanui,PF,-16.4833,-151.7500,79.03,70,24,10.29,few clouds
3,3,Kapaa,US,22.0752,-159.3190,82.72,75,90,5.01,light rain
9,9,Hilo,US,19.7297,-155.0900,80.80,81,90,5.75,light rain
11,11,Buala,SB,-8.1450,159.5921,78.60,81,93,8.43,light rain
17,17,Touros,BR,-5.1989,-35.4608,79.70,73,46,17.81,scattered clouds
22,22,Lorengau,PG,-2.0226,147.2712,80.01,82,100,7.02,overcast clouds
28,28,Cabedelo,BR,-6.9811,-34.8339,77.67,81,24,17.63,few clouds
29,29,Georgetown,MY,5.4112,100.3354,81.90,89,20,2.30,few clouds
32,32,Jingdezhen,CN,29.2947,117.2079,76.93,90,27,4.72,light rain
33,33,Atuona,PF,-9.8000,-139.0333,77.88,75,36,15.90,light rain


In [8]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                186
City                   186
Country                186
Lat                    186
Lng                    186
Max Temp               186
Humidity               186
Cloudiness             186
Wind Speed             186
Current Description    186
dtype: int64

In [9]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.


In [10]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Faanui,PF,79.03,few clouds,-16.4833,-151.7500,
3,Kapaa,US,82.72,light rain,22.0752,-159.3190,
9,Hilo,US,80.80,light rain,19.7297,-155.0900,
11,Buala,SB,78.60,light rain,-8.1450,159.5921,
17,Touros,BR,79.70,scattered clouds,-5.1989,-35.4608,
22,Lorengau,PG,80.01,overcast clouds,-2.0226,147.2712,
28,Cabedelo,BR,77.67,few clouds,-6.9811,-34.8339,
29,Georgetown,MY,81.90,few clouds,5.4112,100.3354,
32,Jingdezhen,CN,76.93,light rain,29.2947,117.2079,
33,Atuona,PF,77.88,light rain,-9.8000,-139.0333,


In [11]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params['location'] = f'{lat},{lng}'
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    try:
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except IndexError:
      print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [12]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.dropna(how = 'any')
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Faanui,PF,79.03,few clouds,-16.4833,-151.7500,Bora-Bora Pearl Beach Resort & Spa
3,Kapaa,US,82.72,light rain,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
9,Hilo,US,80.80,light rain,19.7297,-155.0900,Hilo Hawaiian Hotel
11,Buala,SB,78.60,light rain,-8.1450,159.5921,Maringe Lagoon Lodge
17,Touros,BR,79.70,scattered clouds,-5.1989,-35.4608,INN NEW HORIZON
22,Lorengau,PG,80.01,overcast clouds,-2.0226,147.2712,Lorengau Harbourside Hotel
28,Cabedelo,BR,77.67,few clouds,-6.9811,-34.8339,Hotel Pigalle
29,Georgetown,MY,81.90,few clouds,5.4112,100.3354,Cititel Penang
32,Jingdezhen,CN,76.93,light rain,29.2947,117.2079,Garden Hotel
33,Atuona,PF,77.88,light rain,-9.8000,-139.0333,Villa Enata


In [14]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [18]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [19]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure()
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))